In [1]:
%pip install azureml-widgets
%pip install azureml-train-automl-runtime==1.57.0
%pip install --upgrade azureml-sdk[notebooks,automl]


StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 12, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 78.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 82.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 KB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.4/617.4 KB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 KB 38.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Not uninstalling urllib3 at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-3aa8e68f-35b9-4ea6-97ef-fa49e76c1a17
    Can't uninstall 'urllib3'. No files were found to uninstall.
  Attempting uninstall: isodate
    Found existing installation: isodate 0.6.0
    Not uninstalling isodate at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-package

In [2]:
from azureml.core import Workspace, Experiment

# ws = Workspace.from_config()
subscription_name = "quick-starts-ws-267011"
subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb' #61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30



ws = Workspace.get(name = subscription_name,
subscription_id=subscription_id,
resource_group="aml-quickstarts-267011"
)


exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


run = exp.start_logging()

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 18, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DMMQT2KFL to authenticate.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-267011
Azure region: westeurope
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-267011


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = subscription_name

vm_size = "Standard_D2_V2"
compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
#Create the compute cluster
compute_target = ComputeTarget.create(ws,cluster_name, compute_config)
#wait for the compute ready
compute_target.wait_for_completion(show_output=True)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 19, Finished, Available, Finished)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {"C": uniform(0.05, 0.1),
    "max_iter" : choice(16, 32, 64, 128)}
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.2, evaluation_interval=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', 
file_path='Users/odl_user_267011/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="Users/odl_user_267011/train.py",
    environment=sklearn_env,
    compute_target=compute_target
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 20, Finished, Available, Finished)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

experiment = Experiment(workspace=ws, name="my-hyperdive-experiment")
experiment_run = experiment.submit(hyperdrive_config)
RunDetails(experiment_run).show()
experiment_run.wait_for_completion(show_output=True)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 21, Finished, Available, Finished)

Failed to load image Python extension: libc10_cuda.so: cannot open shared object file: No such file or directory


Initializing logging file for interpret-community


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767
Web View: https://ml.azure.com/runs/HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-267011/workspaces/quick-starts-ws-267011&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-08-25T01:42:33.215832][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-08-25T01:42:33.7307482Z][SCHEDULER][INFO]Scheduling job, id='HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767_0' 
[2024-08-25T01:42:33.8286669Z][SCHEDULER][INFO]Scheduling job, id='HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767_1' 
[2024-08-25T01:42:33.9797020Z][SCHEDULER][INFO]Scheduling job, id='HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767_2' 
[2024-08-25T01:42:34.0919798Z][SCHEDULER][INFO]Scheduling job, id='HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767_3' 
[2024-08-25T01:42:34.019117][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767',
 'target': 'quick-starts-ws-267011',
 'status': 'Completed',
 'startTimeUtc': '2024-08-25T01:42:32.110582Z',
 'endTimeUtc': '2024-08-25T02:01:43.745852Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'e5cdddec-0b74-40d1-a31d-33846fa70445',
  'user_agent': 'python/3.10.6 (Linux-4.15.0-1178-azure-x86_64-with-glibc2.27) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767_7',
  'score': '0.9084977238239758',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767_7'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  '

In [6]:
all_runs = experiment.get_runs()
for run in all_runs:
    print(run.id, run.status)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 22, Finished, Available, Finished)

HD_7384e1b0-e283-4a34-b8b1-d917b9c2f767 Completed


In [7]:
import joblib
# Get your best run and save the model from that run.

best_run = experiment_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 23, Finished, Available, Finished)

{'Regularization Strength:': 0.05259553437493178, 'Max iterations:': 64, 'Accuracy': 0.9084977238239758}


In [8]:
best_run.get_file_names()

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 24, Finished, Available, Finished)

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2024-08-25-02',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [9]:
#Save model
model = best_run.register_model(model_name = "auto_hyper_best_model",
model_path="outputs/model.joblib")

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 25, Finished, Available, Finished)

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 26, Finished, Available, Finished)

In [11]:
from Users.odl_user_267011.train import clean_data

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 27, Finished, Available, Finished)

In [12]:
# from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 28, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '63630bc3-7f0b-4603-adb2-efdb758c34dc'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': '63630bc3-7f0b-4603-adb2-efdb758c34dc', 'run_id': '63630bc3-7f0b-4603-adb2-efdb758c34dc'}


In [13]:
from sklearn.model_selection import train_test_split
import pandas as pd

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 29, Finished, Available, Finished)

In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
y_train = y_train.to_frame()

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 30, Finished, Available, Finished)

In [15]:
y_train.head()

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 31, Finished, Available, Finished)

,y
26465,0
1387,0
22628,0
8336,0
19898,0


In [16]:
training_data = pd.concat([x_train, y_train], axis=1)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 32, Finished, Available, Finished)

In [17]:
training_data.head(3)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 33, Finished, Available, Finished)

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,...,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
26465,52,1,0,1,0,8,3,13,14,999,...,0,0,0,0,1,0,0,0,0,0
1387,41,1,0,1,0,6,4,84,3,999,...,1,1,0,0,0,0,0,0,0,0
22628,27,0,0,0,0,7,1,331,2,999,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
# tabular_dataset = TabularDatasetFactory.register_pandas_dataframe(
#     dataframe=training_data,
#     target=ws.get_default_datastore(),
#     name="my_training_data"
# )

In [18]:
datastore = ws.get_default_datastore()
datastore

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 34, Finished, Available, Finished)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-45fa72f9-7442-48bd-b26f-e0a6a3e47430",
  "account_name": "mlstrg267011",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [22]:
training_data.to_csv("training/training_data.csv", index=False)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 38, Finished, Available, Finished)

In [20]:
from azureml.core import Dataset

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 36, Finished, Available, Finished)

In [23]:
datastore.upload(src_dir='training/', target_path='data/')

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 39, Finished, Available, Finished)

Uploading an estimated of 3 files
Uploading training/training_data.csv
Uploaded training/training_data.csv, 1 files out of an estimated total of 3
Uploading training/.amlignore
Uploaded training/.amlignore, 2 files out of an estimated total of 3
Uploading training/.amlignore.amltmp
Uploaded training/.amlignore.amltmp, 3 files out of an estimated total of 3
Uploaded 3 files


$AZUREML_DATAREFERENCE_16012b873a9f4d50b820256cbd3a33f0

In [24]:
my_data = Dataset.Tabular.from_delimited_files(path=[(datastore, ('data/training_data.csv'))])

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 40, Finished, Available, Finished)

In [25]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=my_data,
    label_column_name='y',
    n_cross_validations=2,
    max_concurrent_iterations=4,
    compute_target=compute_target)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 41, Finished, Available, Finished)

In [26]:
# Submit your automl run

experiment_auto = Experiment(ws, 'automl_exp')
auto_run = experiment_auto.submit(automl_config, show_output=True)
RunDetails(auto_run).show()
auto_run.wait_for_completion(show_output=True)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 42, Finished, Available, Finished)

Submitting remote run.
No run_configuration provided, running on quick-starts-ws-267011 with default configuration
Running on remote compute: quick-starts-ws-267011


Experiment,Id,Type,Status,Details Page,Docs Page
automl_exp,AutoML_01e8101e-e9e0-45af-802c-114354a692af,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: DatasetEvaluation. Generating features for the dataset.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-----------------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl_exp,AutoML_01e8101e-e9e0-45af-802c-114354a692af,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2951                          |True                            |26360                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_01e8101e-e9e0-45af-802c-114354a692af',
 'target': 'quick-starts-ws-267011',
 'status': 'Completed',
 'startTimeUtc': '2024-08-25T02:08:33.414061Z',
 'endTimeUtc': '2024-08-25T02:40:35.427765Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'quick-starts-ws-267011',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_exp","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-267011","workspace_name":"quick-starts-ws-267011","region":"westeurope","compute_target":"q

In [ ]:
# %pip install azureml-train-automl-runtime==1.57.0

In [27]:
# Retrieve and save your best automl model.

best_auto_run, best_auto_model = auto_run.get_output()

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 43, Finished, Available, Finished)

[02:53:18] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:1203: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, please export the model by calling `Booster.save_model` from that version
  first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/latest/tutorials/saving_model.html

  for more details about differences between saving model and serializing.

[02:53:18] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:888: Found JSON model saved before XGBoost 1.6, please save the model using current version again. The support for old JSON model will be discontinued in XGBoost 2.3.
[02:53:18] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1667849527992/work/src/learner.cc:553: 
  If you are loading a serialized model (like pickle in Python, RDS in R) generated by
  older XGBoost, pl

In [29]:
best_auto_run.register_model(model_name="best_auto_model.pkl",
model_path='./outputs/')

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 45, Finished, Available, Finished)

Model(workspace=Workspace.create(name='quick-starts-ws-267011', subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb', resource_group='aml-quickstarts-267011'), name=best_auto_model.pkl, id=best_auto_model.pkl:1, version=1, tags={}, properties={})

In [30]:
print(best_auto_model._final_estimator)

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 46, Finished, Available, Finished)

PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('31', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bylevel=0.6, colsample_bytree=1, eta=0.001, gamma=0, max_depth=6, max_leaves=15, n_estimators=800, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=2.5, subsample=1, tree_method='auto'))], verbose=False)), ('11', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.6, eta=0.3, gamma=0, max_depth=6, max_leaves=0, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_sta

In [31]:
#delete compute_target
compute_target.delete()

StatementMeta(4efcc3c7-516a-47e7-83c2-733fb44882fb, 0, 47, Finished, Available, Finished)